# 2013년부터 2022년까지의 전체 패널 데이터를 대상으로,
- 종속변수: GDP
- 후보 독립변수: Corporate Tax, GERD, Institutions, Internet Usage 조합
- 최적의 모델(AIC 기준)이 선택

In [4]:
import pandas as pd
import numpy as np
import itertools
import math
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

- 로그우도, 표본 수, 추정 파라미터 수를 이용하여 AIC 또는 BIC를 계산

In [5]:
def calculate_criterion(loglik, nobs, k, criterion='AIC'):
    if criterion == 'AIC':
        return -2 * loglik + 2 * k
    elif criterion == 'BIC':
        return -2 * loglik + k * math.log(nobs)
    else:
        raise ValueError("Use 'AIC' or 'BIC'")

- panel_df: MultiIndex (Country, Year) DataFrame, 각 변수는 wide 형식
- dep_candidates: 종속변수 후보 목록 e.g.: ['GDP', 'Unemployment Rate']
- indep_candidates: 독립변수 후보 목록 e.g.: ['Corporate Tax', 'GERD', 'Institutions', 'Internet Usage']
- entity_effects: 고정효과 포함 여부 (If true -> 고정효과 모델)
- criterion: 모델 선택 기준 ('AIC' | 'BIC')
- 가능한 모든 조합을 시도해 기준이 최소인 모델 정보를 반환

In [6]:
def best_subset_panel_all(panel_df, dep_candidates, indep_candidates, entity_effects=True, criterion='AIC'):

    best_model_info = None
    
    # 후보 종속변수와 독립변수의 모든 조합 탐색
    for dep_var in dep_candidates:
        for r in range(1, len(indep_candidates) + 1):
            for combo in itertools.combinations(indep_candidates, r):
                vars_to_use = [dep_var] + list(combo)
                # 결측치 제거
                temp = panel_df[vars_to_use].dropna()
                if temp.empty:
                    continue
                y = temp[dep_var]
                X = temp[list(combo)]
                X = sm.add_constant(X)
                
                try:
                    model = PanelOLS(y, X, entity_effects=entity_effects)
                    res = model.fit(cov_type='clustered', cluster_entity=True)
                    # 파라미터 수: 상수 포함
                    k = len(res.params)
                    score = calculate_criterion(res.loglik, res.nobs, k, criterion=criterion)
                    
                    if best_model_info is None or score < best_model_info['score']:
                        best_model_info = {
                            'dep_var': dep_var,
                            'indep_vars': combo,
                            'score': score,
                            'model': res,
                            'nobs': res.nobs,
                            'criterion': criterion
                        }
                except Exception as e:
                    print(f"Model estimation failed for {dep_var} with independents {combo}. Error: {e}")
    return best_model_info

In [7]:
if __name__ == "__main__":
    # 1. 데이터 로드
    df_master = pd.read_csv("../data/master_data_by_category.csv")
    # long format으로 변환
    years = [str(y) for y in range(2013, 2023)]
    df_long = df_master.melt(id_vars=['Country', 'category'], value_vars=years,
                            var_name='Year', value_name='Value')
    df_long['Year'] = df_long['Year'].astype(int)
    # pivot: 인덱스 = (Country, Year), 컬럼 = category, 값 = Value
    panel_df = df_long.pivot(index=['Country', 'Year'], columns='category', values='Value')
    panel_df.sort_index(level=['Country', 'Year'], inplace=True)
    
    # 2. 후보 변수 설정
    dep_candidates = ['GDP', 'Unemployment Rate']  # 종속변수 후보
    indep_candidates = ['Corporate Tax', 'GERD', 'Institutions', 'Internet Usage']  # 독립변수 후보
    
    # 3. Best Subset Selection 수행
    best_model = best_subset_panel_all(panel_df, dep_candidates, indep_candidates, entity_effects=True, criterion='AIC')
    
    if best_model:
        print("=== Best PanelOLS Model Selected ===")
        print("Dependent Variable:", best_model['dep_var'])
        print("Independent Variables:", best_model['indep_vars'])
        print("Criterion Used:", best_model['criterion'])
        print("Score:", best_model['score'])
        print("Number of Observations:", best_model['nobs'])
        print(best_model['model'].summary)
    else:
        print("No valid model could be estimated.")

=== Best PanelOLS Model Selected ===
Dependent Variable: Unemployment Rate
Independent Variables: ('Corporate Tax', 'GERD')
Criterion Used: AIC
Score: 171.79321908830423
Number of Observations: 70
                          PanelOLS Estimation Summary                           
Dep. Variable:      Unemployment Rate   R-squared:                        0.2197
Estimator:                   PanelOLS   R-squared (Between):              0.3877
No. Observations:                  70   R-squared (Within):               0.2197
Date:                Wed, Mar 12 2025   R-squared (Overall):              0.3559
Time:                        12:51:40   Log-likelihood                   -82.897
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      8.5878
Entities:                           7   P-value                           0.0005
Avg Obs:                      10.0000   Distribution:                    F

## 모든 카테고리를 종속, 독립 변수로 사용

- calculate_criterion 함수 그대로 사용

- panel_df: 인덱스=(Country,Year), 컬럼=모든 category (wide format)
- entity_effects: 고정효과(True) / 없음(False)
- criterion: 'AIC' or 'BIC'

1) 모든 카테고리를 종속변수 후보로 시도
2) 종속변수와 동일한 카테고리는 독립변수 후보에서 제거
3) 독립변수 후보 부분집합(최소 1개 이상)을 탐색
4) PanelOLS 적합 후, AIC/BIC가 최소인 모델 선택

### 아래 코드는 너무 오래 걸림(조합의 수가 多) So, 병렬 처리 필요

In [8]:
# def best_subset_panel_all_categories(panel_df, entity_effects=True, criterion='AIC'):

#     all_cats = list(panel_df.columns)  # 모든 카테고리
#     best_model_info = None
    
#     for dep_var in all_cats:
#         if panel_df[dep_var].dropna().empty:
#             continue
        
#         # 독립변수 후보: 종속변수를 제외한 나머지
#         indep_candidates = [c for c in all_cats if c != dep_var]
        
#         # 최소 1개 이상 독립변수를 선택하는 모든 조합
#         for r in range(1, len(indep_candidates) + 1):
#             for combo in itertools.combinations(indep_candidates, r):
#                 vars_used = [dep_var] + list(combo)
#                 temp = panel_df[vars_used].dropna()
#                 if temp.empty:
#                     continue
                
#                 y = temp[dep_var]
#                 X = temp[list(combo)]
#                 X = sm.add_constant(X)
                
#                 try:
#                     model = PanelOLS(y, X, entity_effects=entity_effects)
#                     res = model.fit(cov_type='clustered', cluster_entity=True)
#                     # 파라미터 수(상수 포함)
#                     k = len(res.params)  
#                     score = calculate_criterion(res.loglik, res.nobs, k, criterion)
                    
#                     if best_model_info is None or score < best_model_info['score']:
#                         best_model_info = {
#                             'dep_var': dep_var,
#                             'indep_vars': combo,
#                             'score': score,
#                             'model': res,
#                             'nobs': res.nobs,
#                             'criterion': criterion
#                         }
#                 except Exception as e:
#                     # 모델 추정 실패 시 pass
#                     pass
#     return best_model_info

In [9]:
# if __name__ == "__main__":
#     df_master = pd.read_csv("../data/master_data_by_category.csv")
#     years = [str(y) for y in range(2013, 2023)]
#     df_long = df_master.melt(
#         id_vars=['Country', 'category'],
#         value_vars=years,
#         var_name='Year',
#         value_name='Value'
#     )
#     df_long['Year'] = df_long['Year'].astype(int)
#     panel_df = df_long.pivot(index=['Country','Year'], columns='category', values='Value')
#     panel_df.sort_index(level=['Country','Year'], inplace=True)
    
#     # 2) Best Subset Selection (모든 카테고리)
#     best_model = best_subset_panel_all_categories(panel_df, entity_effects=True, criterion='AIC')
    
#     if best_model:
#         print("=== Best PanelOLS Model Selected ===")
#         print("Dependent Variable:", best_model['dep_var'])
#         print("Independent Variables:", best_model['indep_vars'])
#         print(f"{best_model['criterion']}: {best_model['score']}")
#         print("Number of Observations:", best_model['nobs'])
#         print(best_model['model'].summary)
#     else:
#         print("No valid model could be estimated.")

### 병렬 처리

In [10]:
from joblib import Parallel, delayed

- 주어진 종속 변수(dep_var)와 독립 변수 조합(combo)에 대해 PanelOLS를 적합하고, 
- AIC | BIC 점수를 계산하는 함수

In [11]:
def evaluate_combo(dep_var, combo, panel_df, entity_effects, criterion):

    vars_to_use = [dep_var] + list(combo)
    temp = panel_df[vars_to_use].dropna()
    if temp.empty:
        return None
    y = temp[dep_var]
    X = temp[list(combo)]
    X = sm.add_constant(X)
    
    try:
        model = PanelOLS(y, X, entity_effects=entity_effects)
        res = model.fit(cov_type='clustered', cluster_entity=True)
        k = len(res.params)  # 상수항 포함 파라미터 수
        score = calculate_criterion(res.loglik, res.nobs, k, criterion)
        return {
            'dep_var': dep_var,
            'indep_vars': combo,
            'score': score,
            'model': res,
            'nobs': res.nobs
        }
    except Exception as e:
        return None

- panel_df: MultiIndex (Country, Year)를 갖는 DataFrame (wide format, 각 변수=category)
- dep_candidates: 종속변수 후보 목록 (예: ['GDP', 'Unemployment Rate', ...])
- indep_candidates: 전체 독립변수 후보 목록 (이 함수에서는 모든 카테고리를 대상,
                    각 종속 변수 후보에 대해 독립변수 후보는 종속 변수를 제외한 모든 변수로 자동 설정)
- entity_effects: 고정효과 포함 여부 (If true -> 고정효과 모델)
- criterion: 모델 선택 기준 ('AIC' 또는 'BIC')
- n_jobs: 병렬 처리 시 사용 CPU 코어 수 (기본 -1: 가능한 모든 코어 사용)
- return: 최적 모델 정보를 담은 dict

In [12]:
def best_subset_panel_all_categories_parallel(panel_df, dep_candidates, indep_candidates, entity_effects=True, criterion='AIC', n_jobs=-1):
    all_cats = list(panel_df.columns)  # 모든 카테고리가 독립변수 후보
    best_model_info = None
    results = []
    
    for dep_var in dep_candidates:
        # 독립 변수 후보: 전체 후보에서 종속 변수를 제거
        indep_candidates_dep = [c for c in all_cats if c != dep_var]
        
        # 가능한 모든 독립 변수 조합 (최소 1개 이상)
        for r in range(1, len(indep_candidates_dep) + 1):
            combos = list(itertools.combinations(indep_candidates_dep, r))
            # 병렬 처리하여 각 조합에 대해 모델 평가
            res_list = Parallel(n_jobs=n_jobs, backend="multiprocessing")(
                delayed(evaluate_combo)(dep_var, combo, panel_df, entity_effects, criterion) for combo in combos
            )
            # None인 결과 제거
            res_list = [res for res in res_list if res is not None]
            results.extend(res_list)
    
    if results:
        best_model_info = min(results, key=lambda x: x['score'])
    return best_model_info

In [ ]:
# if __name__ == "__main__":
#     df_master = pd.read_csv("../data/master_data_by_category.csv")

#     years = [str(y) for y in range(2013, 2023)]
#     df_long = df_master.melt(id_vars=['Country', 'category'], value_vars=years,
#                             var_name='Year', value_name='Value')
#     df_long['Year'] = df_long['Year'].astype(int)

#     # 3. Wide format: 인덱스=(Country, Year), 컬럼=각 category, 값=Value
#     panel_df = df_long.pivot(index=['Country', 'Year'], columns='category', values='Value')
#     panel_df.sort_index(level=['Country', 'Year'], inplace=True)

#     # 4. 후보 변수 설정: 모든 카테고리를 종속 변수 후보
#     dep_candidates = list(panel_df.columns)
#     # 독립 변수 후보는 모두 동일(전체 카테고리), 각 경우에 종속 변수는 자동으로 제거
#     indep_candidates = list(panel_df.columns)

#     # 5. Best Subset Selection 실행 (전체 패널 데이터 사용, 고정효과 모델)
#     best_model = best_subset_panel_all_categories_parallel(
#         panel_df, 
#         dep_candidates=dep_candidates, 
#         indep_candidates=indep_candidates, 
#         entity_effects=True, 
#         criterion='AIC', 
#         n_jobs=-1
#     )

#     if best_model:
#         print("=== Best PanelOLS Model Selected ===")
#         print("Dependent Variable:", best_model['dep_var'])
#         print("Independent Variables:", best_model['indep_vars'])
#         print("Criterion:", best_model['criterion'])
#         print("Score:", best_model['score'])
#         print("Number of Observations:", best_model['nobs'])
#         print(best_model['model'].summary)
#     else:
#         print("No valid model could be estimated.")

KeyboardInterrupt: 